In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName('Caching in RDD').enableHiveSupport().getOrCreate()

25/06/17 06:08:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [4]:
!hadoop fs -ls /data/

Found 5 items
-rw-r--r--   2 root hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 03:55 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 04:03 /data/write_output_1
drwxr-xr-x   - root hadoop          0 2025-06-08 04:09 /data/write_output_2
drwxr-xr-x   - root hadoop          0 2025-06-08 03:58 /data/write_output_another.csv


In [5]:
customers_rdd = spark.sparkContext.textFile('/data/customers_100.csv')

In [6]:
customers_filtered = customers_rdd.filter(lambda row:'Mumbai' in row)
customers_mapped = customers_filtered.map(lambda row:(row.split(',')[0],1))
customers_reduced = customers_mapped.reduceByKey(lambda x,y:x+y) 

In [7]:
import time
start_time = time.time()
customers_reduced.count()
end_time = time.time()
print(f"Time taken : {end_time-start_time} seconds")

Time taken : 3.7087934017181396 seconds


In [8]:
customers_reduced.count()

9

2nd run was very fast as pyspark does caching on its own, we would now cache the data 
#### Caching is by default lazy .

In [9]:
customers_reduced.cache() # nothing would be Shown in the spark UI as caching is lazy.

PythonRDD[8] at RDD at PythonRDD.scala:53

In [10]:
customers_reduced.count()

9

In [11]:
# uncaching
customers_reduced.unpersist()

PythonRDD[8] at RDD at PythonRDD.scala:53

In [12]:
spark.stop()